In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\kandr\appdata\local\continuum\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
!pip install joblib

In [3]:
!pip install tensorflow

  Using cached https://files.pythonhosted.org/packages/c6/dc/9030097e5774fe02d1be3cb42eb54125d2c0607a6c11172f1dcad2b7fdcc/tensorflow-2.3.0-cp37-cp37m-win_amd64.whl


In [4]:
# Dependencies
import numpy as np
import pandas as pd

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
#tensorflow.keras.__version__
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [6]:
import warnings
warnings.simplefilter('ignore')

In [7]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [8]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [9]:
Xtemp = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period','koi_time0bk','koi_slogg','koi_srad','koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr','koi_steff','koi_slogg','koi_srad','ra','dec','koi_kepmag']]
Xtemp

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_slogg,koi_srad,koi_impact,koi_duration,...,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,162.513840,4.467,0.927,0.586,4.50700,...,2.83,443,9.11,25.8,5455,4.467,0.927,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,175.850252,4.544,0.868,0.969,1.78220,...,14.60,638,39.30,76.3,5853,4.544,0.868,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,170.307565,4.564,0.791,1.276,2.40641,...,33.46,1395,891.96,505.6,5805,4.564,0.791,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,171.595550,4.438,1.046,0.701,1.65450,...,2.75,1406,926.16,40.9,6031,4.438,1.046,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,172.979370,4.486,0.972,0.762,3.14020,...,2.77,1160,427.65,40.2,6046,4.486,0.972,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,0,0,0,1,8.589871,132.016100,4.296,1.088,0.765,4.80600,...,1.11,929,176.40,8.4,5638,4.296,1.088,298.74921,46.973351,14.478
6987,0,1,1,0,0.527699,131.705093,4.529,0.903,1.252,3.22210,...,29.35,2088,4500.53,453.3,5638,4.529,0.903,297.18875,47.093819,14.082
6988,0,0,0,0,1.739849,133.001270,4.444,1.031,0.043,3.11400,...,0.72,1608,1585.81,10.6,6119,4.444,1.031,286.50937,47.163219,14.757
6989,0,0,1,0,0.681402,132.181750,4.447,1.041,0.147,0.86500,...,1.07,2218,5713.41,12.3,6173,4.447,1.041,294.16489,47.176281,15.385


In [10]:
y = df[['koi_disposition']]
data_binary_encoded = pd.get_dummies(y, columns=["koi_disposition"])
data_binary_encoded.columns = [["candidate","confirmed","false_positive"]]

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(Xtemp, data_binary_encoded)

In [12]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.12282572, 0.10010297, 0.12252498, 0.04895905, 0.04041636,
       0.02690836, 0.01764624, 0.01731739, 0.04015034, 0.02904395,
       0.05122564, 0.07453804, 0.03210544, 0.03198111, 0.12991152,
       0.02114399, 0.01608106, 0.0173487 , 0.02089611, 0.01924253,
       0.01963051])

In [13]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, Xtemp), reverse=True)

[(0.12991151877210172, 'koi_model_snr'),
 (0.12282572461964013, 'koi_fpflag_nt'),
 (0.12252497944188498, 'koi_fpflag_co'),
 (0.10010296526447064, 'koi_fpflag_ss'),
 (0.07453804375110083, 'koi_prad'),
 (0.05122563848247295, 'koi_depth'),
 (0.04895904569878345, 'koi_fpflag_ec'),
 (0.04041636206141344, 'koi_period'),
 (0.04015034036017476, 'koi_impact'),
 (0.032105439636079945, 'koi_teq'),
 (0.03198110914473922, 'koi_insol'),
 (0.029043945574134717, 'koi_duration'),
 (0.026908361167653116, 'koi_time0bk'),
 (0.02114399031576066, 'koi_steff'),
 (0.0208961066052518, 'ra'),
 (0.01963051283739296, 'koi_kepmag'),
 (0.019242526927912364, 'dec'),
 (0.017646241175743694, 'koi_slogg'),
 (0.017348698680615577, 'koi_srad'),
 (0.017317386890301014, 'koi_srad'),
 (0.01608106259237213, 'koi_slogg')]

In [14]:
X = Xtemp.drop(columns=['ra','dec','koi_kepmag','koi_srad','koi_slogg'])
X

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff
0,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,25.8,5455
1,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,76.3,5853
2,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,5805
3,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,40.9,6031
4,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,2.77,1160,427.65,40.2,6046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,0,0,0,1,8.589871,132.016100,0.765,4.80600,87.7,1.11,929,176.40,8.4,5638
6987,0,1,1,0,0.527699,131.705093,1.252,3.22210,1579.2,29.35,2088,4500.53,453.3,5638
6988,0,0,0,0,1.739849,133.001270,0.043,3.11400,48.5,0.72,1608,1585.81,10.6,6119
6989,0,0,1,0,0.681402,132.181750,0.147,0.86500,103.6,1.07,2218,5713.41,12.3,6173


In [15]:
# Set features. This will also be used as your x values.
selected_features = X.columns
selected_features

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth',
       'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_steff'],
      dtype='object')

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [17]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff
3563,0,0,0,0,10.548413,139.064020,1.0170,1.8720,102.9,3.89,899,154.84,11.7,6047
4099,0,0,0,0,24.754385,140.207320,0.7090,3.3900,593.3,2.10,491,13.70,18.0,4852
5460,0,0,0,0,1.057336,131.792007,0.2620,1.5795,47337.0,14.59,1276,623.51,476.0,4664
1091,0,0,0,0,201.118319,187.569860,0.0010,10.3280,584.8,2.28,300,1.92,34.7,5646
5999,0,0,0,0,91.649983,175.715600,0.2136,10.2940,193.6,2.27,568,24.57,8.7,6705


In [18]:
# Using MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [20]:
#Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [21]:
print(X_train_scaled.shape, y_train_categorical.shape)

(5243, 14) (5243, 3)


In [22]:
# Using StandardScaler
X2_scaler = StandardScaler().fit(X_train)
X2_train_scaled = X2_scaler.transform(X_train)
X2_test_scaled = X2_scaler.transform(X_test)

In [23]:
model = Sequential()
# give no. of columns in y_train_categorical as input_dim
model.add(Dense(units=100, activation='relu', input_dim=14))
model.add(Dense(units=100, activation='relu'))
# give of columns in y_train_categorical as units
model.add(Dense(units=3, activation='softmax'))

In [24]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1500      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 11,903
Trainable params: 11,903
Non-trainable params: 0
_________________________________________________________________


In [25]:
model2 = Sequential()
model2.add(Dense(units=100, activation='relu', input_dim=14))
model2.add(Dense(units=100, activation='relu'))
model2.add(Dense(units=3, activation='softmax'))

In [26]:
# Compile and fit the model
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               1500      
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 11,903
Trainable params: 11,903
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
164/164 - 0s - loss: 0.5434 - accuracy: 0.7028
Epoch 2/60
164/164 - 0s - loss: 0.3934 - accuracy: 0.7822
Epoch 3/60
164/164 - 0s - loss: 0.3828 - accuracy: 0.7864
Epoch 4/60
164/164 - 0s - loss: 0.3788 - accuracy: 0.7896
Epoch 5/60
164/164 - 0s - loss: 0.3746 - accuracy: 0.7934
Epoch 6/60
164/164 - 0s - loss: 0.3693 - accuracy: 0.8058
Epoch 7/60
164/164 - 0s - loss: 0.3685 - accuracy: 0.7984
Epoch 8/60
164/164 - 0s - loss: 0.3659 - accuracy: 0.8062
Epoch 9/60
164/164 - 0s - loss: 0.3612 - accuracy: 0.8097
Epoch 10/60
164/164 - 0s - loss: 0.3600 - accuracy: 0.8100
Epoch 11/60
164/164 - 0s - loss: 0.3575 - accuracy: 0.8110
Epoch 12/60
164/164 - 0s - loss: 0.3546 - accuracy: 0.8150
Epoch 13/60
164/164 - 0s - loss: 0.3533 - accuracy: 0.8161
Epoch 14/60
164/164 - 0s - loss: 0.3484 - accuracy: 0.8245
Epoch 15/60
164/164 - 0s - loss: 0.3492 - accuracy: 0.8150
Epoch 16/60
164/164 - 0s - loss: 0.3453 - accuracy: 0.8209
Epoch 17/60
164/164 - 0s - loss: 0.3397 - accuracy: 0.8262
Epoch 

In [29]:
model_loss, model_accuracy = model.evaluate(
     X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2746 - accuracy: 0.8907
Deep Neural Network - Loss: 0.2745939791202545, Accuracy: 0.8907322883605957


In [30]:
model.fit(
     X_train_scaled,
     y_train_categorical,
     epochs=80,
     shuffle=True,
     verbose=2
 )

Epoch 1/80
164/164 - 0s - loss: 0.2694 - accuracy: 0.8819
Epoch 2/80
164/164 - 0s - loss: 0.2705 - accuracy: 0.8747
Epoch 3/80
164/164 - 0s - loss: 0.2617 - accuracy: 0.8840
Epoch 4/80
164/164 - 0s - loss: 0.2687 - accuracy: 0.8777
Epoch 5/80
164/164 - 0s - loss: 0.2619 - accuracy: 0.8871
Epoch 6/80
164/164 - 0s - loss: 0.2615 - accuracy: 0.8835
Epoch 7/80
164/164 - 0s - loss: 0.2597 - accuracy: 0.8873
Epoch 8/80
164/164 - 0s - loss: 0.2598 - accuracy: 0.8854
Epoch 9/80
164/164 - 0s - loss: 0.2598 - accuracy: 0.8858
Epoch 10/80
164/164 - 0s - loss: 0.2553 - accuracy: 0.8919
Epoch 11/80
164/164 - 0s - loss: 0.2649 - accuracy: 0.8812
Epoch 12/80
164/164 - 0s - loss: 0.2613 - accuracy: 0.8858
Epoch 13/80
164/164 - 0s - loss: 0.2586 - accuracy: 0.8863
Epoch 14/80
164/164 - 0s - loss: 0.2562 - accuracy: 0.8907
Epoch 15/80
164/164 - 0s - loss: 0.2536 - accuracy: 0.8892
Epoch 16/80
164/164 - 0s - loss: 0.2573 - accuracy: 0.8852
Epoch 17/80
164/164 - 0s - loss: 0.2570 - accuracy: 0.8859
Epoch 

In [31]:
model_loss, model_accuracy = model.evaluate(
     X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2748 - accuracy: 0.8896
Deep Neural Network - Loss: 0.27476245164871216, Accuracy: 0.8895881175994873


In [32]:
model.fit(
     X_train_scaled,
     y_train_categorical,
     epochs=100,
     shuffle=True,
     verbose=2
 )

Epoch 1/100
164/164 - 0s - loss: 0.2396 - accuracy: 0.8953
Epoch 2/100
164/164 - 0s - loss: 0.2324 - accuracy: 0.8961
Epoch 3/100
164/164 - 0s - loss: 0.2351 - accuracy: 0.8978
Epoch 4/100
164/164 - 0s - loss: 0.2324 - accuracy: 0.8964
Epoch 5/100
164/164 - 0s - loss: 0.2309 - accuracy: 0.8980
Epoch 6/100
164/164 - 0s - loss: 0.2328 - accuracy: 0.8964
Epoch 7/100
164/164 - 0s - loss: 0.2351 - accuracy: 0.8966
Epoch 8/100
164/164 - 0s - loss: 0.2328 - accuracy: 0.9001
Epoch 9/100
164/164 - 0s - loss: 0.2344 - accuracy: 0.8991
Epoch 10/100
164/164 - 0s - loss: 0.2308 - accuracy: 0.9012
Epoch 11/100
164/164 - 0s - loss: 0.2331 - accuracy: 0.8997
Epoch 12/100
164/164 - 0s - loss: 0.2353 - accuracy: 0.8966
Epoch 13/100
164/164 - 0s - loss: 0.2296 - accuracy: 0.8981
Epoch 14/100
164/164 - 0s - loss: 0.2324 - accuracy: 0.8966
Epoch 15/100
164/164 - 0s - loss: 0.2283 - accuracy: 0.8985
Epoch 16/100
164/164 - 0s - loss: 0.2312 - accuracy: 0.8972
Epoch 17/100
164/164 - 0s - loss: 0.2292 - accura

In [33]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2655 - accuracy: 0.9010
Deep Neural Network - Loss: 0.2655249536037445, Accuracy: 0.9010297656059265


In [34]:
# This one scores better than epochs 60,80,and 100
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=150,
    shuffle=True,
    verbose=2
)

Epoch 1/150
164/164 - 0s - loss: 0.2181 - accuracy: 0.9079
Epoch 2/150
164/164 - 0s - loss: 0.2198 - accuracy: 0.9043
Epoch 3/150
164/164 - 0s - loss: 0.2160 - accuracy: 0.9046
Epoch 4/150
164/164 - 0s - loss: 0.2150 - accuracy: 0.9064
Epoch 5/150
164/164 - 0s - loss: 0.2142 - accuracy: 0.9035
Epoch 6/150
164/164 - 0s - loss: 0.2137 - accuracy: 0.9041
Epoch 7/150
164/164 - 0s - loss: 0.2178 - accuracy: 0.9037
Epoch 8/150
164/164 - 0s - loss: 0.2224 - accuracy: 0.9056
Epoch 9/150
164/164 - 0s - loss: 0.2166 - accuracy: 0.9043
Epoch 10/150
164/164 - 0s - loss: 0.2135 - accuracy: 0.9046
Epoch 11/150
164/164 - 0s - loss: 0.2163 - accuracy: 0.9041
Epoch 12/150
164/164 - 0s - loss: 0.2121 - accuracy: 0.9060
Epoch 13/150
164/164 - 0s - loss: 0.2117 - accuracy: 0.9075
Epoch 14/150
164/164 - 0s - loss: 0.2149 - accuracy: 0.9044
Epoch 15/150
164/164 - 0s - loss: 0.2115 - accuracy: 0.9092
Epoch 16/150
164/164 - 0s - loss: 0.2119 - accuracy: 0.9060
Epoch 17/150
164/164 - 0s - loss: 0.2159 - accura

164/164 - 0s - loss: 0.1954 - accuracy: 0.9144
Epoch 138/150
164/164 - 0s - loss: 0.2008 - accuracy: 0.9130
Epoch 139/150
164/164 - 0s - loss: 0.2002 - accuracy: 0.9128
Epoch 140/150
164/164 - 0s - loss: 0.1941 - accuracy: 0.9153
Epoch 141/150
164/164 - 0s - loss: 0.1971 - accuracy: 0.9117
Epoch 142/150
164/164 - 0s - loss: 0.1970 - accuracy: 0.9163
Epoch 143/150
164/164 - 0s - loss: 0.1974 - accuracy: 0.9132
Epoch 144/150
164/164 - 0s - loss: 0.1946 - accuracy: 0.9163
Epoch 145/150
164/164 - 0s - loss: 0.1984 - accuracy: 0.9134
Epoch 146/150
164/164 - 0s - loss: 0.1956 - accuracy: 0.9163
Epoch 147/150
164/164 - 0s - loss: 0.1973 - accuracy: 0.9138
Epoch 148/150
164/164 - 0s - loss: 0.1941 - accuracy: 0.9176
Epoch 149/150
164/164 - 0s - loss: 0.2015 - accuracy: 0.9092
Epoch 150/150
164/164 - 0s - loss: 0.1957 - accuracy: 0.9142


In [35]:
# Testing with StandardScaler
model2.fit(
    X2_train_scaled,
    y_train_categorical,
    epochs=150,
    shuffle=True,
    verbose=2
)

Epoch 1/150
164/164 - 0s - loss: 0.4984 - accuracy: 0.7625
Epoch 2/150
164/164 - 0s - loss: 0.3691 - accuracy: 0.8114
Epoch 3/150
164/164 - 0s - loss: 0.3535 - accuracy: 0.8161
Epoch 4/150
164/164 - 0s - loss: 0.3446 - accuracy: 0.8198
Epoch 5/150
164/164 - 0s - loss: 0.3388 - accuracy: 0.8251
Epoch 6/150
164/164 - 0s - loss: 0.3350 - accuracy: 0.8240
Epoch 7/150
164/164 - 0s - loss: 0.3290 - accuracy: 0.8255
Epoch 8/150
164/164 - 0s - loss: 0.3300 - accuracy: 0.8211
Epoch 9/150
164/164 - 0s - loss: 0.3226 - accuracy: 0.8306
Epoch 10/150
164/164 - 0s - loss: 0.3200 - accuracy: 0.8318
Epoch 11/150
164/164 - 0s - loss: 0.3207 - accuracy: 0.8356
Epoch 12/150
164/164 - 0s - loss: 0.3188 - accuracy: 0.8341
Epoch 13/150
164/164 - 0s - loss: 0.3135 - accuracy: 0.8367
Epoch 14/150
164/164 - 0s - loss: 0.3110 - accuracy: 0.8356
Epoch 15/150
164/164 - 0s - loss: 0.3103 - accuracy: 0.8386
Epoch 16/150
164/164 - 0s - loss: 0.3073 - accuracy: 0.8402
Epoch 17/150
164/164 - 0s - loss: 0.3076 - accura

164/164 - 0s - loss: 0.1808 - accuracy: 0.9226
Epoch 138/150
164/164 - 0s - loss: 0.1881 - accuracy: 0.9130
Epoch 139/150
164/164 - 0s - loss: 0.1886 - accuracy: 0.9165
Epoch 140/150
164/164 - 0s - loss: 0.1827 - accuracy: 0.9180
Epoch 141/150
164/164 - 0s - loss: 0.1856 - accuracy: 0.9182
Epoch 142/150
164/164 - 0s - loss: 0.1849 - accuracy: 0.9201
Epoch 143/150
164/164 - 0s - loss: 0.1874 - accuracy: 0.9176
Epoch 144/150
164/164 - 0s - loss: 0.1851 - accuracy: 0.9149
Epoch 145/150
164/164 - 0s - loss: 0.1838 - accuracy: 0.9186
Epoch 146/150
164/164 - 0s - loss: 0.1807 - accuracy: 0.9197
Epoch 147/150
164/164 - 0s - loss: 0.1810 - accuracy: 0.9193
Epoch 148/150
164/164 - 0s - loss: 0.1803 - accuracy: 0.9195
Epoch 149/150
164/164 - 0s - loss: 0.1773 - accuracy: 0.9226
Epoch 150/150
164/164 - 0s - loss: 0.1763 - accuracy: 0.9229


In [36]:
# MinMaxScaler Accuracy
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.3282 - accuracy: 0.8856
Deep Neural Network - Loss: 0.3281726539134979, Accuracy: 0.8855835199356079


In [37]:
# StandardScaler
model_loss, model_accuracy = model2.evaluate(
    X2_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.4709 - accuracy: 0.8833
Deep Neural Network - Loss: 0.4708515703678131, Accuracy: 0.8832951784133911


In [38]:
# make predictions and print the results
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {y_test.values[:5].tolist()}")

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Predicted classes: ['CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CONFIRMED'
 'FALSE POSITIVE']
Actual Labels: [['CONFIRMED'], ['FALSE POSITIVE'], ['FALSE POSITIVE'], ['CONFIRMED'], ['FALSE POSITIVE']]


Deep Learning with MinMaxScaler scored higher accuracy percentage (88.56%) then StandardScaler (88.32%). 